<a href="https://colab.research.google.com/github/sandroklippel/misc_noteboooks/blob/main/Historico_Mapbiomas_de_um_arquivo_GeoJSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Histórico de Uso e Cobertura do Solo segundo Mapbiomas para um determinado polígono (`.geojson`)

Instruções:

1. Gere um arquivo `map.geojson` com a área desejada (polígono no SRC EPSG:4326). Pode ser criado no QGIS, ArcGIS ou no site https://geojson.io/.
2. Carregue esse arquivo no seu Google Drive. Se já houver um arquivo anterior, sobrescrever.
3. Selecione no menu "Ambiente de execução" -> "Executar tudo" ou pressione Ctrl+F9
4. Na primeira execução será necessário autorizar a conexão com o Google Drive. Siga as instruções.
5. Pode ser necessário fazer a autenticação do Earth Engine. Siga as instruções, copie e cole o código no local indicado e tecle [ENTER].
6. O gráfico será gerado na última célula. Há opção para salvá-lo em .PNG.



In [ ]:
import json
from google.colab import drive

drive.mount('/gdrive', force_remount=True)

with open('/gdrive/MyDrive/map.geojson') as f:
  feature = json.load(f)


In [ ]:
import ee

try:
  ee.Initialize()
except ee.ee_exception.EEException:
  ee.Authenticate()
  ee.Initialize()

In [ ]:
import datetime
import pandas as pd
import numpy as np
import plotly.express as px

def map_function(s):
  def mbyr(y):
      k = ee.String('classification_').cat(ee.String(y))
      d = ee.Dictionary(s.get(k))
      return d.set('year', y)
  return mbyr

classes = {3: "Formação Florestal",
           4: "Formação Savânica",
           5: "Mangue",
           9: "Floresta Plantada",
           11: "Área Úmida não Florestal",
           12: "Formação Campestre",
           13: "Outra Formação Natural não Florestal",
           15: "Pastagem",
           18: "Agricultura",
           19: "Cultura Anual e Perene",
           20: "Cultura Semi-Perene",
           21: "Mosaico de Agricultura e Pastagem",
           23: "Praia e Duna",
           24: "Infraestrutura Urbana",
           25: "Outra Área não Vegetada",
           27: "Não observado",
           29: "Afloramento Rochoso",
           30: "Mineração",
           31: "Aquicultura",
           33: "Rio, Lago e Oceano",
           32: "Apicum",
           34: "Glacias",
           36: "Lavoura Perene",
           39: "Soja",
           40: "Arroz",
           41: "Outras Lavouras Temporárias",
           46: "Café",
           47: "Citrus",
           48: " Outras Lavouras Perene",
           49: "Restinga Arborizad"}

cores = {3: "#274e13",
         4: "#32cd32",
         5: "#687537",
         9: "#935132",
        11: "#45c2a5",
        12: "#b8af4f",
        13: "#bdb76b",
        15: "#ffd966",
        18: "#e974ed",
        19: "#d5a6bd",
        20: "#c27ba0",
        21: "#ffefc3",
        23: "#dd7e6b",
        24: "#af2a2a",
        25: "#ff99ff",
        27: "#d5d5e5",
        29: "#ff8c00",
        30: "#8a2be2",
        31: "#29eee4",
        33: "#0000ff",
        32: "#968c46",
        34: "#968c46",
        36: "#f3b4f1",
        39: "#e075ad",
        40: "#982c9e",
        41: "#e787f8",
        46: "#cca0d4",
        47: "#d082de",
        48: "#cd49e4",
        49: "6b9932"}

color_map = {classes[k]: v for k, v in cores.items()}

hoje = datetime.datetime.today().strftime('%d/%m/%Y')

geom = ee.FeatureCollection(feature).geometry()
area_ha = geom.area().getInfo() / 10000.0

mapbiomas = ee.Image('projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1')

stats = (mapbiomas.reduceRegion(
         reducer = ee.Reducer.frequencyHistogram(),
        geometry = geom,
        scale = 30,
        crs = 'EPSG:4326'
        ))

years = ee.List([str(i) for i in range(1985,2021)])

map_by_year = map_function(stats)

lst = years.map(map_by_year)

df = pd.DataFrame(lst.getInfo())
df.set_index('year', inplace=True)
df_perc = df.div(df.iloc[0,:].sum(), axis=1).mul(100).round(2)
df_perc.columns = [classes[int(c)] for c in df_perc.columns.tolist()]

In [ ]:
fig = px.bar(df_perc, barmode='stack', 
             color_discrete_map=color_map, 
             labels={"value": "% da área do polígono", "variable": "", "year": ""},
             title=f"Histórico de Uso e Cobertura do Solo ({area_ha:.2f} ha)")

fig.update_layout(template='simple_white')
fig.update_xaxes(tickangle=-90)
fig.add_annotation(
    text = f"Fonte dos dados: Projeto MapBiomas – Coleção 6 da Série Anual de Mapas de Uso e Cobertura da Terra do Brasil, acesso em {hoje}.",
    showarrow=False,
    x = 0,
    y = -0.2,
    xref='paper',
    yref='paper',
    xanchor='left',
    yanchor='bottom',
    xshift=-1,
    yshift=-5,
    font=dict(size=10, color="grey"),
    align="left")
fig.show()